# **SETUP**

## Import

In [3]:
from __future__ import division
import pickle
import os
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

## Init

In [4]:
seed = 76424236
np.random.seed(seed=seed)

folder_name = f"./generated_data/"
if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

## Load training set

In [5]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name+"training_arr.npy")
raw_data['test_data_pos'] = np.load(folder_name+"unbiased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load(folder_name+"unbiased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

## Define Model

In [6]:
import tensorflow as tf                         # Code to avoid tf using cached embeddings
tf.compat.v1.reset_default_graph()

cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

cml_model.load(folder_name+"cml-yahoo")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






2024-04-24 10:13:43.513101: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-04-24 10:13:43.515987: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 4192050000 Hz
2024-04-24 10:13:43.516573: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56340cd7a700 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-24 10:13:43.516591: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


INFO:tensorflow:Restoring parameters from ./generated_data/cml-yahoo


## Generate Raw Results

In [7]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-pos-unbiased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-neg-unbiased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 2296/2296 [00:01<00:00, 1625.42it/s]


{'AUC': [0.4077777777777778,
  0.46399999999999997,
  0.5833333333333334,
  0.5927777777777777,
  0.4261111111111111,
  0.29571428571428576,
  0.33416666666666667,
  0.495,
  0.4075,
  0.5044444444444445,
  0.410625,
  0.4857142857142857,
  0.4828571428571428,
  0.46437500000000004,
  0.45562500000000006,
  0.5883333333333334,
  0.42000000000000004,
  0.43499999999999994,
  0.5471428571428572,
  0.4905555555555556,
  0.501875,
  0.4675000000000001,
  0.4116666666666667,
  0.50125,
  0.606875,
  0.37625,
  0.2,
  0.445625,
  0.306875,
  0.63,
  0.47214285714285714,
  0.5816666666666667,
  0.3638888888888889,
  0.5038888888888889,
  0.4861111111111111,
  0.42714285714285716,
  0.48062499999999997,
  0.2075,
  0.6461111111111111,
  0.4035714285714286,
  0.54,
  0.33625,
  0.639375,
  0.5959999999999999,
  0.4288888888888889,
  0.42312500000000003,
  0.5549999999999999,
  0.4066666666666667,
  0.59,
  0.4311111111111111,
  0.42000000000000004,
  0.483,
  0.3025,
  0.3333333333333333,
  0.3

# **DEFINE FUNCTION**

In [8]:
def eq(infilename, infilename_neg, trainfilename, gamma=1.0, K=1):

    # Load pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    
    # Merge the two dictionaries
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    
    Zui = dict()
    Ni = dict()

    # Count item frequencies in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Count the number of users with at least one positive item in the test set 
    # (not the smartest way)
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break

    # Compute the recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute the scores using AUC and compute the recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [9]:
def aoa(infilename, infilename_neg, trainfilename, K=1):

    # Load pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    
    # Merge the two dictionaries
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
        
    Zui = dict()
    Ni = dict()
    
    # Count item frequencies in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    
    # Count the number of users with at least one positive item in the test set 
    # (not the smartest way)
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break

    # Compute the recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
            
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute the scores using AUC and compute the recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser]))
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0
            denominator += 1 
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

# **TEST**

## Compute results

In [10]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5)

{'auc': 0.8439872027026072, 'recall': 0.05204219465712629}

In [11]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2)

{'auc': 0.8398511463738416, 'recall': 0.049855378908885875}

In [12]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3)

{'auc': 0.8345355174867299, 'recall': 0.047191065066928835}

In [13]:
aoa(folder_name+"cml-yahoo-test-pos-unbiased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy")

{'auc': 0.7382109673137573, 'recall': 0.3108651692384276}